In [1]:
# Dataset Info.

# train [폴더] :
# train.csv [파일]
# 2018년 ~ 2021년의 모든 농산물의 상세 정보 및 가격
# meta [폴더]
# TRAIN_산지공판장_2018-2021.csv : 전국도매 경락정보
# TRAIN_전국도매_2018-2021.csv : 산지공판장 경락정보


# test [폴더] :
# 2022년의 특정 시점 T가 비식별화된 평가 데이터
# 예측의 입력으로 사용될 예측 시점 T를 포함한 최대 3개월의 9개의 시점에 대한 데이터 (총 25개의 평가 데이터셋 샘플 구성)
# TEST_00.csv ~ TEST_24.csv [파일]
# meta [폴더]
# TEST_전국도매_00.csv ~ TEST_전국도매_24.csv [파일]
# TEST_산지공판장_00.csv ~ TEST_산지공판장_24.csv [파일]
# 				※ 예시) TEST_00 추론 시에는 TEST_00.csv. 필요시에는 TEST_전국도매_00.csv와 TEST_산지공판장_00.csv만 추론에 사용 가능



# sample_submission.csv [ 파일 ] :
# 각 품목의 TEST 파일별 +1순, +2순, +3순의 평균가격(원) 예측 결과
# 시점 : TEST_00+1순, TEST_00+2순, TEST_00+3순 ... TEST_24+1순, TEST_24+2순, TEST_24+3순


# 예측 대상의 품목,품종,등급 상세 :


INTRODUCTION

In [2]:
# 농산물 가격 예측을 위한 AI 모델 개발
# '2024 농산물 가격 예측 AI 경진대회'는 데이터와 AI 기술을 활용하여 농산물 가격 예측 능력을 향상시키는 것을 목표로 합니다.
# 이 대회는 농업 분야의 복잡한 시계열 데이터를 효율적으로 분석하고 예측할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다.

# 이 대회의 궁극적 목적은 참가자들의 시계열 데이터 분석 및 예측 역량을 강화하고,
# AI 기술이 실제 농산물 가격 예측과 관련 정책 결정에 어떻게 기여할 수 있는지 탐구하는 것입니다.

In [3]:
# Import Library
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from types import SimpleNamespace
from sklearn.preprocessing import MinMaxScaler
import os

In [4]:
# Hyperparameter Setting
config = {
    "learning_rate": 2e-5,
    "epoch": 30,
    "batch_size": 64,
    "hidden_size": 64,
    "num_layers": 2,
    "output_size": 3
}

CFG = SimpleNamespace(**config)

품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']

In [5]:
# Define Function for Feature Engineering
# 타겟의 필터 조건을 제외한 메타데이터의 필터 조건은 참가자들 각자의 기준에 맞춰 자유롭게 사용가능
# 밑의 필터 조건은 임의로 제공하는 예시
def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, scaler=None):
    raw_data = pd.read_csv(raw_file)
    산지공판장 = pd.read_csv(산지공판장_file)
    전국도매 = pd.read_csv(전국도매_file)

    # 타겟 및 메타데이터 필터 조건 정의
    conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None, 
        '도매': None  
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }

    # 타겟 데이터 필터링
    raw_품목 = raw_data[raw_data['품목명'] == 품목명]
    target_mask = conditions[품목명]['target'](raw_품목)
    filtered_data = raw_품목[target_mask]

    # 다른 품종에 대한 파생변수 생성
    other_data = raw_품목[~target_mask]
    unique_combinations = other_data[['품종명', '거래단위', '등급']].drop_duplicates()
    for _, row in unique_combinations.iterrows():
        품종명, 거래단위, 등급 = row['품종명'], row['거래단위'], row['등급']
        mask = (other_data['품종명'] == 품종명) & (other_data['거래단위'] == 거래단위) & (other_data['등급'] == 등급)
        temp_df = other_data[mask]
        for col in ['평년 평균가격(원)', '평균가격(원)']:
            new_col_name = f'{품종명}_{거래단위}_{등급}_{col}'
            filtered_data = filtered_data.merge(temp_df[['시점', col]], on='시점', how='left', suffixes=('', f'_{new_col_name}'))
            filtered_data.rename(columns={f'{col}_{new_col_name}': new_col_name}, inplace=True)


    # 공판장 데이터 처리
    if conditions[품목명]['공판장']:
        filtered_공판장 = 산지공판장
        for key, value in conditions[품목명]['공판장'].items():
            filtered_공판장 = filtered_공판장[filtered_공판장[key].isin(value)]
        
        filtered_공판장 = filtered_공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')

    # 도매 데이터 처리
    if conditions[품목명]['도매']:
        filtered_도매 = 전국도매
        for key, value in conditions[품목명]['도매'].items():
            filtered_도매 = filtered_도매[filtered_도매[key].isin(value)]
        
        filtered_도매 = filtered_도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # 수치형 컬럼 처리
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data = filtered_data[['시점'] + list(numeric_columns)]
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0)

    # 정규화 적용
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler

In [6]:
# Define Custom Dataset Class
class AgriculturePriceDataset(Dataset):
    def __init__(self, dataframe, window_size=9, prediction_length=3, is_test=False):
        self.data = dataframe
        self.window_size = window_size
        self.prediction_length = prediction_length
        self.is_test = is_test
        
        self.price_column = '평균가격(원)' 
        self.numeric_columns = self.data.select_dtypes(include=[np.number]).columns.tolist()
        
        self.sequences = []
        if not self.is_test:
            for i in range(len(self.data) - self.window_size - self.prediction_length + 1):
                x = self.data[self.numeric_columns].iloc[i:i+self.window_size].values
                y = self.data[self.price_column].iloc[i+self.window_size:i+self.window_size+self.prediction_length].values
                self.sequences.append((x, y))
        else:
            self.sequences = [self.data[self.numeric_columns].values]
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        if not self.is_test:
            x, y = self.sequences[idx]
            return torch.FloatTensor(x), torch.FloatTensor(y)
        else:
            return torch.FloatTensor(self.sequences[idx])

In [7]:
# Define Model and Training Functions
class PricePredictionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PricePredictionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)  

def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    return total_loss / len(test_loader)

In [8]:
# Train Models and Generate Predictions
품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    train_data, scaler = process_data("./train/train.csv", 
                              "./train/meta/TRAIN_산지공판장_2018-2021.csv", 
                              "./train/meta/TRAIN_전국도매_2018-2021.csv", 
                              품목명)
    품목별_scalers[품목명] = scaler
    dataset = AgriculturePriceDataset(train_data)

    # 데이터를 train과 validation으로 분할
    train_data, val_data = train_test_split(dataset, test_size=0.2, random_state=42)
    
    train_loader = DataLoader(train_data, CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_data, CFG.batch_size, shuffle=False)

    input_size = len(dataset.numeric_columns)
    
    model = PricePredictionLSTM(input_size, CFG.hidden_size, CFG.num_layers, CFG.output_size)
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), CFG.learning_rate)
    
    best_val_loss = float('inf')
    os.makedirs('models', exist_ok=True)

    for epoch in range(CFG.epoch):
        train_loss = train_model(model, train_loader, criterion, optimizer, CFG.epoch)
        val_loss = evaluate_model(model, val_loader, criterion)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'models/best_model_{품목명}.pth')
        
        print(f'Epoch {epoch+1}/{CFG.epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    print(f'Best Validation Loss for {품목명}: {best_val_loss:.4f}')
    
    품목_predictions = []

    ### 추론 
    pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
    for i in pbar_inner:
        test_file = f"./test/TEST_{i:02d}.csv"
        산지공판장_file = f"./test/meta/TEST_산지공판장_{i:02d}.csv"
        전국도매_file = f"./test/meta/TEST_전국도매_{i:02d}.csv"
        
        test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명])
        test_dataset = AgriculturePriceDataset(test_data, is_test=True)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_loader:
                output = model(batch)
                predictions.append(output.numpy())
        
        predictions_array = np.concatenate(predictions)

        # 예측값을 원래 스케일로 복원
        price_column_index = test_data.columns.get_loc(test_dataset.price_column)
        predictions_reshaped = predictions_array.reshape(-1, 1)
        
        # 가격 열에 대해서만 inverse_transform 적용
        price_scaler = MinMaxScaler()
        price_scaler.min_ = 품목별_scalers[품목명].min_[price_column_index]
        price_scaler.scale_ = 품목별_scalers[품목명].scale_[price_column_index]
        predictions_original_scale = price_scaler.inverse_transform(predictions_reshaped)
        #print(predictions_original_scale)
        
        if np.isnan(predictions_original_scale).any():
            pbar_inner.set_postfix({"상태": "NaN"})
        else:
            pbar_inner.set_postfix({"상태": "정상"})
            품목_predictions.extend(predictions_original_scale.flatten())

            
    품목별_predictions[품목명] = 품목_predictions
    pbar_outer.update(1)

품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.4332, Val Loss: 0.3423
Epoch 2/30, Train Loss: 0.4265, Val Loss: 0.3417
Epoch 3/30, Train Loss: 0.4151, Val Loss: 0.3410
Epoch 4/30, Train Loss: 0.4202, Val Loss: 0.3403
Epoch 5/30, Train Loss: 0.4234, Val Loss: 0.3397
Epoch 6/30, Train Loss: 0.4243, Val Loss: 0.3390
Epoch 7/30, Train Loss: 0.4219, Val Loss: 0.3383
Epoch 8/30, Train Loss: 0.4131, Val Loss: 0.3377
Epoch 9/30, Train Loss: 0.4158, Val Loss: 0.3370
Epoch 10/30, Train Loss: 0.4215, Val Loss: 0.3363
Epoch 11/30, Train Loss: 0.4224, Val Loss: 0.3357
Epoch 12/30, Train Loss: 0.4087, Val Loss: 0.3350
Epoch 13/30, Train Loss: 0.4241, Val Loss: 0.3343
Epoch 14/30, Train Loss: 0.4186, Val Loss: 0.3336
Epoch 15/30, Train Loss: 0.4120, Val Loss: 0.3329
Epoch 16/30, Train Loss: 0.4112, Val Loss: 0.3323
Epoch 17/30, Train Loss: 0.4172, Val Loss: 0.3316
Epoch 18/30, Train Loss: 0.3981, Val Loss: 0.3309
Epoch 19/30, Train Loss: 0.4131, Val Loss: 0.3302
Epoch 20/30, Train Loss: 0.4203, Val Loss: 0.3295
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.4757, Val Loss: 0.4151
Epoch 2/30, Train Loss: 0.4943, Val Loss: 0.4140
Epoch 3/30, Train Loss: 0.4811, Val Loss: 0.4128
Epoch 4/30, Train Loss: 0.4854, Val Loss: 0.4117
Epoch 5/30, Train Loss: 0.4658, Val Loss: 0.4106
Epoch 6/30, Train Loss: 0.4777, Val Loss: 0.4095
Epoch 7/30, Train Loss: 0.4775, Val Loss: 0.4084
Epoch 8/30, Train Loss: 0.4843, Val Loss: 0.4073
Epoch 9/30, Train Loss: 0.4686, Val Loss: 0.4061
Epoch 10/30, Train Loss: 0.4758, Val Loss: 0.4050
Epoch 11/30, Train Loss: 0.4808, Val Loss: 0.4039
Epoch 12/30, Train Loss: 0.4685, Val Loss: 0.4028
Epoch 13/30, Train Loss: 0.4742, Val Loss: 0.4016
Epoch 14/30, Train Loss: 0.4669, Val Loss: 0.4005
Epoch 15/30, Train Loss: 0.4652, Val Loss: 0.3994
Epoch 16/30, Train Loss: 0.4678, Val Loss: 0.3983
Epoch 17/30, Train Loss: 0.4602, Val Loss: 0.3971
Epoch 18/30, Train Loss: 0.4565, Val Loss: 0.3960
Epoch 19/30, Train Loss: 0.4663, Val Loss: 0.3949
Epoch 20/30, Train Loss: 0.4673, Val Loss: 0.3937
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

...

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.3156, Val Loss: 0.2615
Epoch 2/30, Train Loss: 0.3044, Val Loss: 0.2605
Epoch 3/30, Train Loss: 0.3058, Val Loss: 0.2595
Epoch 4/30, Train Loss: 0.3111, Val Loss: 0.2585
Epoch 5/30, Train Loss: 0.3135, Val Loss: 0.2575
Epoch 6/30, Train Loss: 0.3002, Val Loss: 0.2565
Epoch 7/30, Train Loss: 0.3062, Val Loss: 0.2555
Epoch 8/30, Train Loss: 0.3069, Val Loss: 0.2545
Epoch 9/30, Train Loss: 0.3068, Val Loss: 0.2535
Epoch 10/30, Train Loss: 0.2991, Val Loss: 0.2525
Epoch 11/30, Train Loss: 0.3053, Val Loss: 0.2515
Epoch 12/30, Train Loss: 0.2975, Val Loss: 0.2505
Epoch 13/30, Train Loss: 0.2992, Val Loss: 0.2495
Epoch 14/30, Train Loss: 0.2981, Val Loss: 0.2485
Epoch 15/30, Train Loss: 0.2999, Val Loss: 0.2474
Epoch 16/30, Train Loss: 0.2999, Val Loss: 0.2464
Epoch 17/30, Train Loss: 0.2971, Val Loss: 0.2454
Epoch 18/30, Train Loss: 0.2905, Val Loss: 0.2443
Epoch 19/30, Train Loss: 0.3000, Val Loss: 0.2433
Epoch 20/30, Train Loss: 0.2906, Val Loss: 0.2422
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/30, Train Loss: 0.3085, Val Loss: 0.2816
Epoch 2/30, Train Loss: 0.3092, Val Loss: 0.2804
Epoch 3/30, Train Loss: 0.3106, Val Loss: 0.2792
Epoch 4/30, Train Loss: 0.3044, Val Loss: 0.2780
Epoch 5/30, Train Loss: 0.3095, Val Loss: 0.2768
Epoch 6/30, Train Loss: 0.2966, Val Loss: 0.2756
Epoch 7/30, Train Loss: 0.3004, Val Loss: 0.2744
Epoch 8/30, Train Loss: 0.3034, Val Loss: 0.2732
Epoch 9/30, Train Loss: 0.2958, Val Loss: 0.2720
Epoch 10/30, Train Loss: 0.3010, Val Loss: 0.2708
Epoch 11/30, Train Loss: 0.2969, Val Loss: 0.2696
Epoch 12/30, Train Loss: 0.2973, Val Loss: 0.2684
Epoch 13/30, Train Loss: 0.3012, Val Loss: 0.2672
Epoch 14/30, Train Loss: 0.2897, Val Loss: 0.2660
Epoch 15/30, Train Loss: 0.2983, Val Loss: 0.2648
Epoch 16/30, Train Loss: 0.2942, Val Loss: 0.2636
Epoch 17/30, Train Loss: 0.2923, Val Loss: 0.2624
Epoch 18/30, Train Loss: 0.2904, Val Loss: 0.2612
Epoch 19/30, Train Loss: 0.2856, Val Loss: 0.2600
Epoch 20/30, Train Loss: 0.2833, Val Loss: 0.2588
Epoch 21/

테스트 파일 추론 중:   0%|          | 0/25 [00:00<?, ?it/s]

In [9]:
# Prepare Submission File
sample_submission = pd.read_csv('./sample_submission.csv')

for 품목명, predictions in 품목별_predictions.items():
    sample_submission[품목명] = predictions

# 결과 저장
sample_submission.to_csv('./baseline_submission.csv', index=False)